## Imports

In [5]:
import numpy as np
import pandas as pd
import talib as ta
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

## Data

In [6]:
# Gets the data of the prices of stock for a specific company
# Adds Movement info based on closing price
# If price does not change we keep movement as up for simplicity sake

path = '../processeddata/individual_stocks_5yr/individual_stocks_5yr/'
ticker = 'AMZN' # Change Stock to analyze

p = path + ticker + '_data.csv' 
df = pd.read_csv(p)
df.set_index(['date'],inplace=True)

df['Movement'] = 'Up'

prevPrice = 0.0
for index, row in df.iterrows():
    if row['close'] < prevPrice:
        df.loc[index, 'Movement'] = 'Down'
    prevPrice = row['close']

print(df)

               open     high       low    close    volume  Name Movement
date                                                                    
2013-02-08   261.40   265.25   260.555   261.95   3879078  AMZN       Up
2013-02-11   263.20   263.25   256.600   257.21   3403403  AMZN     Down
2013-02-12   259.19   260.16   257.000   258.70   2938660  AMZN       Up
2013-02-13   261.53   269.96   260.300   269.47   5292996  AMZN       Up
2013-02-14   267.37   270.65   265.400   269.24   3462780  AMZN     Down
...             ...      ...       ...      ...       ...   ...      ...
2018-02-01  1445.00  1459.88  1385.140  1390.00   9113808  AMZN     Down
2018-02-02  1477.39  1498.00  1414.000  1429.95  11125722  AMZN       Up
2018-02-05  1402.62  1458.98  1320.720  1390.00  11494985  AMZN     Down
2018-02-06  1361.46  1443.99  1351.790  1442.84  11066819  AMZN       Up
2018-02-07  1449.00  1460.99  1415.150  1416.78   7162741  AMZN     Down

[1259 rows x 7 columns]


## Decision Tree Model

In [ ]:
# Setting Technical Indicators
# Momentum: Relative Strength Index (RSI)
# Trend: Moving Average Convergence Divergence (MACD), Estimated Moving Average (EMA)
# Volatility: Average True Range (ATR) 

